In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [4]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/save.png?raw=true' height="50" alt="netdata"/></center>
import os
from IPython.display import clear_output

url = "https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-fp16.ckpt" #@param {type:"string"}

save_models_dir = "/content/sample_data"#@param {type:"string"}
os.chdir(save_models_dir)

!wget --no-check-certificate $url $save_models_dir

clear_output()
print('[1;32mModel Save ✓')

Model Save ✓


In [21]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/install.png?raw=true' height="50" alt="netdata"/></center>
import os
from IPython.utils import capture
from IPython.display import clear_output

SDZ_install = "/content/sample_data" #@param {type:"string"}
os.chdir(SDZ_install)

SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')


with capture.capture_output() as cap:
  gitclo = "git clone --depth 1" 
  !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui
  %cd stable-diffusion-webui
  !COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py
  %cd stable-diffusion-webui

#!wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
!wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css

!git pull


# Install Extensions
os.chdir(SDZ_webui_path)

%cd extensions
!git clone https://github.com/Extraltodeus/depthmap2mask.git
!git clone https://github.com/d8ahazard/sd_dreambooth_extension.git

# Replace path deppmap fix
with open(SDZ_webui_path+'/'+'extensions/depthmap2mask/scripts/depth2image_depthmask.py', 'r') as file :
  filedata = file.read()

filedata = filedata.replace('extensions/depthmap2mask/scripts/depthmap_for_depth2img.py', SDZ_webui_path+'/'+'extensions/depthmap2mask/scripts/depth2image_depthmask.py')

with open(SDZ_webui_path+'/'+'extensions/depthmap2mask/scripts/depth2image_depthmask.py', 'w') as file:
  file.write(filedata)

%cd ..
!mkdir repositories
%cd repositories
!mkdir midas
%cd midas
!git clone https://github.com/isl-org/MiDaS.git .

clear_output()
print('[1;32mReady for SDZ ✓')

Ready for SDZ ✓


In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ2.png?raw=true' height="50" alt="netdata"/></center>
import os
#SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ = "/stable-diffusion-webui"
os.chdir(SDZ_install+SDZ)
models_dir = "/content/sample_data" #@param {type:"string"}
model = "/content/sample_data/v1-5-fp16.ckpt" #@param {type:"string"}
!COMMANDLINE_ARGS="--ckpt-dir=$models_dir --xformers --theme=dark --ckpt=$model --share --api --cors-allow-origins=https://www.painthua.com" REQS_FILE="requirements.txt" python launch.py


In [48]:
!pip install -U diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
